# GenRest tutorial
This tutorial uses synthetic data to demonstrate the genetic stratifier.

In [ ]:
import numpy as np
import pandas as pd
from genrest.genetic_stratifier import GeneticStratifier

In [ ]:
rng = np.random.default_rng(0)
x1 = rng.normal(0,1,1000)
x2 = rng.normal(5,2,1000)
y = x1*0.5 + x2*0.2 + rng.normal(0,1,1000)
data = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})
data.head()

In [ ]:
stratifier = GeneticStratifier(strat_columns=['x1','x2'], target_col='y', population_size=10, generations=5, random_state=0)
best = stratifier.fit(data)
best, stratifier.best_score_